[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Hyper-FLUX-jupyter/blob/main/Hyper_FLUX_8_Steps_Lora_jupyter.ipynb)

In [ ]:
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ByteDance/Hyper-SD/resolve/main/Hyper-FLUX.1-dev-16steps-lora.safetensors -d /content/models -o Hyper-FLUX.1-dev-16steps-lora.safetensors
!pip install xformers==0.0.27.post2 diffusers peft

import torch
from diffusers import FluxPipeline
import random, time

torch.backends.cuda.matmul.allow_tf32 = True
pipe = FluxPipeline.from_pretrained("camenduru/FLUX.1-dev-diffusers", torch_dtype=torch.bfloat16)
pipe.load_lora_weights("/content/models/Hyper-FLUX.1-dev-16steps-lora.safetensors")
pipe.fuse_lora(lora_scale=0.125)
pipe.enable_sequential_cpu_offload()

In [ ]:
prompt = "cute cat"
seed = 0
if seed == 0:
    random.seed(int(time.time()))
    seed = random.randint(0, 18446744073709551615)
steps = 16
scales = 3.5
height = 1024
width = 1024

image = pipe(prompt=[prompt],
    generator=torch.Generator().manual_seed(int(seed)),
    num_inference_steps=int(steps),
    guidance_scale=float(scales),
    height=int(height),
    width=int(width),
    max_sequence_length=256
).images[0]

print(seed)
image